In [6]:
import pandas as pd
import seaborn as sns; sns.set(color_codes=True)
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
import json
# from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn.metrics import silhouette_score

from utils import (
    clean_data, 
    create_pivot, 
    add_metadata_to_pivot,
    get_linkage,
    get_cluster_map,
    correlation_matrix,
    join_on_ska_labels,
    wrap_clustermap_and_mismatches,
    hierarchical_clustering,
    get_silhouette_score,
)

In [7]:
# with location metadata:

new_metadata = pd.read_csv(
    "/Users/phoenix.logan/code/skeeters/data/metadata/CMS001_CMS002_MergedAnnotations.csv", 
    header = 0, 
#     index_col = "czbiohub-mosquito_sequences_id",
)

species_id = new_metadata[[
    "czbiohub-mosquito_sequences_id",
    "visual_genus", 
    "visual_species", 
    "sourmash_species", 
    "sourmash_genus"
]]

species_id.rename(
    columns = {
#         'visual_genus': 'genus',
#         'visual_species': 'species',
        'compute_species': 'sourmash_species',
        'compute_genus': 'sourmash_genus',
        'czbiohub-mosquito_sequences_id': ''
    }, 
    inplace = True
)
species_id.set_index("", inplace=True)

len(species_id) # 161
species_id.head()

,visual_genus,visual_species,sourmash_species,sourmash_genus
,,,,
CMS_001_RNA_A_S1,Culex,erythrothorax,erythrothorax,Culex
CMS_002_RNA_A_S1,Culex,tarsalis,tarsalis,Culex
CMS_003_RNA_A_S2,Culiseta,particeps,particeps,Culiseta
CMS_004_RNA_A_S2,Culex,pipiens,pipiens,Culex
CMS_005_RNA_A_S3,Culiseta,incidens,incidens,Culiseta


In [8]:
from collections import namedtuple
from numpy import nan

import seaborn as sns
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist, squareform
from scipy.spatial.distance import pdist, squareform
import plotly.graph_objects as go
import plotly.figure_factory as ff
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="mosquito_locator")

final_pivot_w_labels, cluster_map, linkage = join_on_ska_labels(
    "s3://phoenixlogan-sketches/k15_10000000.distances.tsv",
    species_id,
    "_001_10000000_15",
    10
)

new_metadata = pd.read_csv(
    "/Users/phoenix.logan/code/skeeters/data/metadata/CMS001_CMS002_MergedAnnotations.csv", 
    header = 0, 
    index_col = "czbiohub-mosquito_sequences_id",
)

counties = [
    'Alameda County',
    'San Diego County',
    'San Bernardino County',
    'Riverside County',
    'Placer County',
    'Sacramento County',
] 

all_metadata = pd.merge(
    final_pivot_w_labels.reset_index()[[
        "Sample_1", 
        "visual_genus", 
        "visual_species", 
        "sourmash_genus",
        "sourmash_species",
        "ska_genus",
        "ska_species"
    ]],
    new_metadata.drop([
        'NewIDseqName', 
        'OldIDseqName',
#         'raw_sequence_run_directory',
        'visual_genus',
        'visual_species',
#         'compute_genus',
#         'compute_species'
        "sourmash_genus",
        "sourmash_species",
        "ska_genus",
        "ska_species"
    ], axis=1),
    how='left',
    left_on="Sample_1", 
    right_index=True
).set_index("Sample_1")


def get_location(row):
    lat = row['collection_lat']
    long = row['collection_long']
    if np.isnan(lat) or np.isnan(long):
        return np.nan
    else:
        print(f"finding location for {lat}, {long}")
        loc = geolocator.reverse(f"{lat}, {long}", timeout=1000)
        return loc.address
    
    
def find_location(row):
    lat = row['collection_lat']
    long = row['collection_long']
    
    if np.isnan(lat) or np.isnan(long):
        return np.nan
    else:
        return unique_dict[(lat, long)]
    
    
def get_county(row):
    address = row['location_address']
    
    if type(address) != str:
        return np.nan
    
    else:
        if 'Alameda' in address:
            return 'Alameda'

        elif 'San Diego' in address:
            return 'San Diego'

        elif 'San Bernardino' in address:
            return 'San Bernardino'

        elif 'Riverside' in address:
            return 'Riverside'

        elif 'Placer' in address:
            return 'Placer'

        elif 'Sacramento' in address:
            return 'Sacramento'

        else:
            return 'Other'

        
def get_region(row):
    
    county = row['county']
    northern_counties = [
        'Alameda',
        'Placer',
        'Sacramento'
    ]
    southern_counties = [
        'San Bernardino',
        'Riverside', 
        'San Diego'
    ]
    
    if type(county) != str:
        return np.nan
    
    else:
        if county in northern_counties:
            return 'Northern California'
        
        elif county in southern_counties:
            return 'Southern California'
        
        else:
            return 'Other'
        

# unique_locs = [(x[0], x[1]) for x in all_metadata[['collection_lat', 'collection_long']].values]

# unique_dict = {
#     i: get_location(i[0], i[1])
#     for i in set(unique_locs)
# }
        
all_metadata['location_address'] = all_metadata.apply(get_location, axis=1)
all_metadata['county'] = all_metadata.apply(get_county, axis=1)
all_metadata['region'] = all_metadata.apply(get_region, axis=1)

label_dicts = [
    all_metadata[i]
    for i in all_metadata.keys()
]

# label_dicts

all_metadata_pivot = pd.merge(
    final_pivot_w_labels.reset_index(),
    all_metadata.reset_index()[[
        "Sample_1",
        "sex",
        "county",
        "region",
        "life_stage"
    ]],
    how='left',
    left_on="Sample_1", 
    right_on="Sample_1",
).set_index([
    "Sample_1", 
    "visual_genus", 
    "visual_species",
    "sourmash_genus",
    "sourmash_species",
    "ska_genus",
    "ska_species",
    "sex",
    "county",
    "region",
    "life_stage"
])
all_metadata_pivot.head()

all_meta_lookup = all_metadata.reset_index().set_index("Sample_1").to_dict('index')



/Users/phoenix.logan/anaconda3/envs/ska/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



finding location for 37.55697, -122.07938
finding location for 32.7981529, -116.9544084
finding location for 32.7996185, -116.93932029999999
finding location for 32.7152395, -117.11484240000001
finding location for 32.5542907, -117.04828
finding location for 32.5542907, -117.04828
finding location for 32.5542907, -117.04828
finding location for 32.5542907, -117.04828
finding location for 32.5542907, -117.04828
finding location for 32.6952215, -117.1221922
finding location for 32.6952215, -117.1221922
finding location for 32.760258, -117.0813933
finding location for 32.7962659, -116.9594824
finding location for 32.940749100000005, -117.21652390000001
finding location for 32.940749100000005, -117.21652390000001
finding location for 32.940749100000005, -117.21652390000001
finding location for 32.940749100000005, -117.21652390000001
finding location for 32.940749100000005, -117.21652390000001
finding location for 32.940749100000005, -117.21652390000001
finding location for 32.5527830000000

In [9]:
# fill in the missing data
all_meta_lookup['CMS_058_RNA_A_S9']['visual_species'] ='erythrothorax'
all_meta_lookup['CMS_058_RNA_A_S9']['visual_genus'] ='Culex'
all_meta_lookup['CMS_058_RNA_A_S9']['county'] =''
all_meta_lookup['CMS_058_RNA_A_S9']['collection_date'] =''
all_meta_lookup['CMS_058_RNA_A_S9']['collection_lat'] =''
all_meta_lookup['CMS_058_RNA_A_S9']['collection_long'] =''
all_meta_lookup['CMS_058_RNA_A_S9']['region'] =''


all_meta_lookup['CMS_037_RNA_A_S21']['county'] =''
all_meta_lookup['CMS_037_RNA_A_S21']['collection_lat'] =''
all_meta_lookup['CMS_037_RNA_A_S21']['collection_long'] =''
all_meta_lookup['CMS_037_RNA_A_S21']['region'] =''

all_meta_lookup['CMS_032_RNA_A_S7']['collection_date'] =''

In [10]:


def add_node(node, parent ):
    # First create the new node and append it to its parent's children
    attributes = {}
    if node.id < len(id2name):
        sample = id2name[node.id]
        attributes["sample_name"] = sample
        attributes["visual_species"] = all_meta_lookup[sample]["visual_species"]
        attributes["ska_species"] = all_meta_lookup[sample]["ska_species"]
        attributes["div"] = node.dist
#         attributes["region"] = all_meta_lookup[sample]["region"]
        attributes["county"] = all_meta_lookup[sample]["county"]
        attributes["date"] = all_meta_lookup[sample]["collection_date"]
#         attributes["latitude"] = all_meta_lookup[sample]["collection_lat"]
#         attributes["longitude"] = all_meta_lookup[sample]["collection_long"]
        
    newNode = dict(
        aa_muts={},
        attr=attributes,
        branch_length=node.dist,
        clock_length=node.dist,
        node_id=node.id,
        clade=node.id,
        muts=[],
        strain="",
        num_date="",
        
    )

    if node.left and node.right:        
        newNode["children"]=[]
    
    parent["children"].append( newNode )

    # Recursively add the current node's children
    if node.left: add_node( node.left, newNode )
    if node.right: add_node( node.right, newNode )

labels = list(final_pivot_w_labels.index.get_level_values(0))
id2name = dict(zip(range(len(labels)), labels))
cdist = scipy.spatial.distance.squareform(final_pivot_w_labels)
Z = shc.linkage(cdist, method="ward")
T = shc.to_tree(Z , rd=False )
d3Dendro = dict(
        aa_muts={},
        attr={},
        branch_length=0,
        clock_length=0,
        clade=0,
        muts=[],
        children=[],
        strain=""
)
add_node( T, d3Dendro )

In [11]:
from collections import defaultdict

palette = [
            "#4c72a5",
            "#48a365",
            "#d0694a",
            "#e1c72f",
            "#cc79a7",
            "#77bedb",
            "#7f6e85",
            "#ccc197",
            "#87CEFF",
            "#3b9072",
            "#b09977",
            "#f05129",
            "#922428",
            "#f5bd42"
]  

def create_entries(fields):
    all_entries = {}
    for field in fields:
        entry = {
            "color_map" : [
                [c, p] for c, p in zip(
                    all_metadata[field].unique(),
                    palette
                )
            ],
            "key": field,
            "legendTitle": field,
            "menuItem": field
        }
        all_entries[field] = entry
    return all_entries
    
    
def create_location(locs):
    loc_entries = {}
    for loc in locs:
        lat = all_metadata[
            all_metadata["county"] == loc
        ][['collection_lat']].mean()[0]

        long = all_metadata[
            all_metadata["county"] == loc
        ][['collection_long']].mean()[0]
        
        loc_entries[loc] = {
            "latitude": lat,
            "longitude": long
        }
        
    return loc_entries


def create_entries(fields):
    all_entries = {}
    for field in fields:
        entry = {
            "color_map" : [
                [c, p] for c, p in zip(
                    all_metadata[field].unique(),
                    palette
                )
            ],
            "key": field,
            "legendTitle": field,
            "menuItem": field,
            "type": "discrete"
        }
        all_entries[field] = entry
    return all_entries
    
meta_dict = {
    "annotations": dict(),
    "author_info": {
        "Phoenix Logan": {"n":366}
    },
    "color_options": create_entries(["county", "visual_species", "ska_species"]),
    "defaults": {
        "colorBy": "visual_species",
        "geoResolution": "county"
    },
    "filters": ["county", "visual_species", "ska_species"],
    "geo": {
        "county": create_location(
            ['Alameda', 'San Diego', 'San Bernardino', 
             'Riverside', 'Placer','Sacramento']
        )
    },
    "maintainer": [
        "Phoenix Logan",
    ],
    "panels": [
        "tree",
        "map", 
    ],
    "title": "SKA species corrections",
    "updated": "October 15th 2019"
}

config_dict = {
  "title": "Ska nextstrain",
  "color_options": {
    "visual_species": {
      "menuItem": "visual_species",
      "legendTitle": "Visual Species",
      "type": "discrete",
      "key": "visual_species"
    },
    "ska_species": {
      "menuItem": "ska_species",
      "legendTitle": "Ska Corrected Species",
      "type": "discrete",
      "key": "ska_species"
    },
    "county": {
      "key":"county",
      "legendTitle":"County",
      "menuItem":"county",
      "type":"discrete"
    },
  },
  "geo": [
    "county"
  ],
  "defaults": {
    "colorBy": "ska_species",
    "geoResolution": "county"
  },
  "maintainer": [
    "Phoenix Logan",
  ],
  "filters": [
    "county",
    "visual_species",
    "ska_species"
  ]
}    

In [12]:
json.dump(d3Dendro, open(
    "/Users/phoenix.logan/code/nextstrain_ska/auspice/nextstrain_ska_tree.json", "w"
), sort_keys=True, indent=4)

json.dump(meta_dict, open(
    "/Users/phoenix.logan/code/nextstrain_ska/auspice/nextstrain_ska_meta.json", "w"
), sort_keys=True, indent=4)

json.dump(config_dict, open(
    "/Users/phoenix.logan/code/nextstrain_ska/config/auspice_config.json", "w"
), sort_keys=True, indent=4)